In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import glob
root_dataset=os.path.join("..")
dataset_dir=os.path.join(root_dataset,"dataset")
dataset_train_path=glob.glob(os.path.join(dataset_dir,"test","*.csv"))

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer

In [4]:
import transformers

In [5]:
import json
import os
from pathlib import Path
law_dir=os.path.join(root_dataset,"wcx_ptt_export_280524")
law_list=glob.glob(os.path.join(law_dir,"*.json"))
law_data = {}
for i in law_list:
    key=Path(i).stem
    law_data[key]={}
    with open(i) as json_file:
        json_data = json.load(json_file)
    for j in json_data:
        law_data[key][j['section_num']]=j['section_content']

In [6]:
model_id="./results_mark4/checkpoint-225"
model_root="unsloth/Meta-Llama-3.1-8B-Instruct"

In [7]:
from datasets import Dataset,DatasetDict
import pandas as pd
import datetime
df = pd.concat([pd.read_csv(i) for i in dataset_train_path])
ds = Dataset.from_pandas(df)
dataset=ds

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_root, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["question"]
    outputs      = examples["answer"]
    list_law= [i for i in examples["relevant_laws"]]
    # list_txt_law=[]
    # for i in list_law:
    #     list_txt_law.append(law_data[i["law_code"][str(i['sections'][0])]])
    texts = []
    for i, output,law in zip(inputs, outputs,list_law):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        list_txt_law=[]
        for k in eval(law):
            if k["law_code"] not in set(law_data.keys()):
                pass
            else:
                if str(k['sections'][0]) not in set(law_data[k["law_code"]].keys()):
                    pass
                else:
                    for s in k['sections']:
                        if str(s) in set(law_data[k["law_code"]].keys()):
                            list_txt_law.append(law_data[k["law_code"]][str(s)].strip())
        if list_txt_law!=[]:
            source_information="- "+"\n- ".join(list_txt_law)
            source_information=source_information.strip()
            i=i.strip()
            content=f"""Query: {i}\nContinue to answer the query by using the Search Results:\n{source_information}"""
            text = tokenizer.apply_chat_template([{"role":"user","content":content}], tokenize=False) + EOS_TOKEN
            texts.append(text)
        else:
            texts.append(None)
    return { "text" : texts, }

from datasets import load_dataset
dataset = dataset.map(formatting_prompts_func, batched = True,).filter(lambda example: isinstance(example['text'],str))

Map:   0%|          | 0/1281 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1281 [00:00<?, ? examples/s]

In [9]:
dataset

Dataset({
    features: ['Unnamed: 0', 'idx', 'question', 'question_ori', 'answer', 'answer_ori', 'relevant_laws', 'irrelevant_laws', 'comment_1_1', 'comment_1_2', 'text'],
    num_rows: 1280
})

In [10]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,#model_root,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    return_full_text=False,
    batch_size=32
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
q="ถ้ามีคนประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตต้องระวางโทษอย่างไร"
source_information="""- พระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546 มาตรา 132 ผู้ใดประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตหรือไม่ได้จดทะเบียนตามมาตรา 54 ต้องระวางโทษจำคุกไม่เกินสามปี หรือปรับไม่เกินสามแสนบาท หรือทั้งจำทั้งปรับ และปรับอีกไม่เกินวันละหนึ่งหมื่นบาทตลอดเวลาที่ยังฝ่าฝืน"""
content=f"""Query: {q}\nContinue to answer the query by using the Search Results:\n{source_information}"""

In [12]:
messages=[{"role":"user","content":content}]
prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipeline(prompt, max_new_tokens=1024, do_sample=False)  # Don't change do_sample=False

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [13]:
messages

[{'role': 'user',
  'content': 'Query: ถ้ามีคนประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตต้องระวางโทษอย่างไร\nContinue to answer the query by using the Search Results:\n- พระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546 มาตรา 132 ผู้ใดประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตหรือไม่ได้จดทะเบียนตามมาตรา 54 ต้องระวางโทษจำคุกไม่เกินสามปี หรือปรับไม่เกินสามแสนบาท หรือทั้งจำทั้งปรับ และปรับอีกไม่เกินวันละหนึ่งหมื่นบาทตลอดเวลาที่ยังฝ่าฝืน'}]

In [14]:
print(prompt)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Query: ถ้ามีคนประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตต้องระวางโทษอย่างไร
Continue to answer the query by using the Search Results:
- พระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546 มาตรา 132 ผู้ใดประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตหรือไม่ได้จดทะเบียนตามมาตรา 54 ต้องระวางโทษจำคุกไม่เกินสามปี หรือปรับไม่เกินสามแสนบาท หรือทั้งจำทั้งปรับ และปรับอีกไม่เกินวันละหนึ่งหมื่นบาทตลอดเวลาที่ยังฝ่าฝืน<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [15]:
outputs[0]['generated_text']

'ตามพระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546 มาตรา 132 ผู้ประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตหรือไม่ได้จดทะเบียนตามมาตรา 54 จะต้องระวางโทษจำคุกไม่เกิน 3 ปี หรือปรับไม่เกิน 300,000 บาท หรือทั้งจำทั้งปรับ และปรับอีกไม่เกิน 1,000 บาทต่อวันตลอดเวลาที่ยังฝ่าฝืน'

In [16]:
from tqdm.auto import tqdm

In [17]:
from transformers.pipelines.pt_utils import KeyDataset

In [18]:
pred_list=[]
for prompt in pipeline(KeyDataset(dataset,"text"), max_new_tokens=1024, do_sample=False):
    # outputs = pipeline(prompt, max_new_tokens=1024, do_sample=False)
    pred_list.extend(prompt)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [19]:
len(pred_list)

1280

In [20]:
pred_list[0]

{'generated_text': 'assistant\n\nตามพระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546 มาตรา 132 ผู้ประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตหรือไม่ได้จดทะเบียนตามมาตรา 54 จะต้องระวางโทษจำคุกไม่เกิน 3 ปี หรือปรับไม่เกิน 300,000 บาท หรือทั้งจำทั้งปรับ และปรับอีกไม่เกิน 1,000 บาทต่อวันตลอดเวลาที่ยังฝ่าฝืน'}

In [21]:
dataset["text"][1]

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nQuery: ถ้าผู้ิยู่ในปกครองได้ยินยอมในการกระทำของผู้ปกครองจะทำให้ผู้ปกครองหลุดพ้นจากความรับผิดหรือเปล่า\nContinue to answer the query by using the Search Results:\n- ประมวลกฎหมายแพ่งและพาณิชย์ มาตรา 1598/5\nถ้าผู้อยู่ในปกครองรู้จักผิดชอบและมีอายุไม่ต่ำกว่าสิบห้าปีบริบูรณ์เมื่อผู้ปกครองจะทำกิจการใดที่สำคัญ ให้ปรึกษาหารือผู้อยู่ในปกครองก่อนเท่าที่จะทำได้\nการที่ผู้อยู่ในปกครองได้ยินยอมด้วยนั้นหาคุ้มผู้ปกครองให้พ้นจากความรับผิดไม่<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n<|eot_id|>'

In [22]:
pred_list[1]

{'generated_text': 'assistant\n\nไม่'}

In [23]:
pred_list[2]

{'generated_text': 'assistant\n\nตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา 876 ผู้เอาประกันภัยสามารถบอกเลิกสัญญาประกันภัยได้หากผู้รับประกันภัยถูกตัดสินให้เป็นคนล้มละลาย แต่หากเบี้ยประกันภัยได้ส่งแล้วเต็มจำนวนเพื่ออายุประกันภัยเป็นระยะเวลามากน้อยเท่าใด ผู้รับประกันภัยจะห้ามบอกเลิกสัญญาก่อนระยะเวลานั้นสุดลง'}

In [24]:
pred_list[3]

{'generated_text': 'assistant\n\nตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา 1030 ถ้าผู้เป็นหุ้นส่วนให้กรรมสิทธิ์ในทรัพย์สินอันใดอันหนึ่งเป็นการลงหุ้น ความเกี่ยวพันระหว่างผู้เป็นหุ้นส่วนคนนั้นกับห้างหุ้นส่วนในเรื่องส่งมอบและซ่อมแซมก็ดี ความรับผิดเพื่อชำรุดบกพร่องก็ดี ความรับผิดเพื่อการรอนสิทธิก็ดี ข้อยกเว้นความรับผิดก็ดี ท่านให้บังคับตามบทบัญญัติแห่งประมวลกฎหมายนี้ ว่าด้วยซื้อขาย'}

In [25]:
pred_list[4]

{'generated_text': 'assistant\n\nตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา 849 การรับเงินหรือรับชำระหนี้อันจะพึงชำระตามสัญญานั้น ท่านให้สันนิษฐานไว้ก่อนว่านายหน้าย่อมไม่มีอำนาจที่จะรับแทนผู้เป็นคู่สัญญา'}

In [26]:
pred_list[40]

{'generated_text': 'assistant\n\nตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา 1424 ผู้ทรงสิทธิเก็บกินมีหน้าที่บำรุงรักษาปกติและซ่อมแซมเล็กน้อยด้วย และหากจำเป็นต้องซ่อมแซมใหญ่หรือมีการสำคัญอันต้องทำเพื่อรักษาทรัพย์สิน ผู้ทรงสิทธิเก็บกินจะต้องแจ้งให้เจ้าของทรัพย์สินทราบโดยพลัน และยอมให้จัดทำการนั้น ๆ ไป หากเจ้าของทรัพย์สินละเลยเสีย ผู้ทรงสิทธิเก็บกินสามารถจัดทำการนั้นไปโดยให้เจ้าของทรัพย์สินออกค่าใช้จ่ายได้\n\nดังนั้น ในกรณีที่โรงงานมีรอยรั่วขนาดใหญ่ ผู้ทรงสิทธิเก็บกินจะต้องแจ้งให้เจ้าของโรงงานทราบโดยพลัน และยอมให้จัดทำการซ่อมแซมทันที หากเจ้าของโรงงานละเลยเสีย ผู้ทรงสิทธิเก็บกินสามารถจัดทำการซ่อมแซมโดยให้เจ้าของโรงงานออกค่าใช้จ่ายได้'}

In [27]:
pred_list[40]["generated_text"].strip("assistant\n\n")

'ตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา 1424 ผู้ทรงสิทธิเก็บกินมีหน้าที่บำรุงรักษาปกติและซ่อมแซมเล็กน้อยด้วย และหากจำเป็นต้องซ่อมแซมใหญ่หรือมีการสำคัญอันต้องทำเพื่อรักษาทรัพย์สิน ผู้ทรงสิทธิเก็บกินจะต้องแจ้งให้เจ้าของทรัพย์สินทราบโดยพลัน และยอมให้จัดทำการนั้น ๆ ไป หากเจ้าของทรัพย์สินละเลยเสีย ผู้ทรงสิทธิเก็บกินสามารถจัดทำการนั้นไปโดยให้เจ้าของทรัพย์สินออกค่าใช้จ่ายได้\n\nดังนั้น ในกรณีที่โรงงานมีรอยรั่วขนาดใหญ่ ผู้ทรงสิทธิเก็บกินจะต้องแจ้งให้เจ้าของโรงงานทราบโดยพลัน และยอมให้จัดทำการซ่อมแซมทันที หากเจ้าของโรงงานละเลยเสีย ผู้ทรงสิทธิเก็บกินสามารถจัดทำการซ่อมแซมโดยให้เจ้าของโรงงานออกค่าใช้จ่ายได้'

In [28]:
from pythainlp.tokenize import word_tokenize

def wordcut(sent):
    return word_tokenize(sent,keep_whitespace=False)

In [29]:
def wordcut2(sent):
    return ' '.join(word_tokenize(sent,keep_whitespace=False))

In [30]:
list_pred_all=[wordcut(i["generated_text"].strip("assistant\n\n")) for i in pred_list]

In [31]:
list_pred_all[0:9]

[['ตาม',
  'พระราชบัญญัติ',
  'สัญญาซื้อขาย',
  'ล่วงหน้า',
  'พ.ศ.',
  '2546',
  'มาตรา',
  '132',
  'ผู้',
  'ประกอบ',
  'กิจการ',
  'ใน',
  'ลักษณะ',
  'เป็น',
  'ศูนย์',
  'ซื้อขาย',
  'สัญญาซื้อขาย',
  'ล่วงหน้า',
  'โดย',
  'ไม่',
  'ได้รับ',
  'ใบอนุญาต',
  'หรือไม่',
  'ได้',
  'จดทะเบียน',
  'ตาม',
  'มาตรา',
  '54',
  'จะ',
  'ต้อง',
  'ระวางโทษ',
  'จำคุก',
  'ไม่',
  'เกิน',
  '3',
  'ปี',
  'หรือ',
  'ปรับ',
  'ไม่',
  'เกิน',
  '300,000',
  'บาท',
  'หรือ',
  'ทั้ง',
  'จำ',
  'ทั้ง',
  'ปรับ',
  'และ',
  'ปรับ',
  'อีก',
  'ไม่',
  'เกิน',
  '1,000',
  'บาท',
  'ต่อ',
  'วัน',
  'ตลอดเวลา',
  'ที่',
  'ยัง',
  'ฝ่าฝืน'],
 ['ไม่'],
 ['ตาม',
  'ประมวลกฎหมายแพ่งและพาณิชย์',
  'มาตรา',
  '876',
  'ผู้เอาประกันภัย',
  'สามารถ',
  'บอกเลิก',
  'สัญญา',
  'ประกันภัย',
  'ได้',
  'หาก',
  'ผู้รับประกันภัย',
  'ถูก',
  'ตัดสิน',
  'ให้',
  'เป็น',
  'คน',
  'ล้มละลาย',
  'แต่',
  'หาก',
  'เบี้ยประกันภัย',
  'ได้',
  'ส่ง',
  'แล้ว',
  'เต็ม',
  'จำนวน',
  'เพื่อ',
  'อายุ',
  'ป

In [32]:
list_ans_all=[wordcut(i) for i in list(dataset["answer"])]

In [33]:
list_ans_all[0:9]

[['ต้อง',
  'ระวางโทษ',
  'จำคุก',
  'ไม่',
  'เกิน',
  'สาม',
  'ปี',
  'หรือ',
  'ปรับ',
  'ไม่',
  'เกิน',
  'สาม',
  'แสน',
  'บาท',
  'หรือ',
  'ทั้ง',
  'จำ',
  'ทั้ง',
  'ปรับ',
  'และ',
  'ปรับ',
  'อีก',
  'ไม่',
  'เกิน',
  'วัน',
  'ละ',
  'หนึ่ง',
  'หมื่น',
  'บาท',
  'ตลอดเวลา',
  'ที่',
  'ยัง',
  'ฝ่าฝืน'],
 ['การ',
  'ที่',
  'ผู้',
  'อยู่',
  'ใน',
  'ปกครอง',
  'ได้',
  'ยินยอม',
  'ด้วย',
  'นั้น',
  'ไม่',
  'ได้',
  'คุ้มครอง',
  'ผู้ปกครอง',
  'ให้',
  'พ้น',
  'จาก',
  'ความ',
  'รับผิด'],
 ['ผู้เอาประกันภัย',
  'จะ',
  'เรียก',
  'ให้หา',
  'ประกัน',
  'อัน',
  'สมควร',
  'ให้',
  'แก่',
  'ตน',
  'ก็ได้',
  'หรือ',
  'จะ',
  'บอกเลิก',
  'สัญญา',
  'ก็ได้'],
 ['ให้',
  'บังคับ',
  'ตาม',
  'บทบัญญัติ',
  'แห่ง',
  'ประมวลกฎหมายแพ่งและพาณิชย์',
  'ว่าด้วย',
  'ซื้อขาย'],
 ['รับ',
  'ชำระหนี้',
  'อัน',
  'จะ',
  'ต้อง',
  'ชำระ',
  'ตาม',
  'สัญญา',
  'นั้น',
  'ให้',
  'สันนิษฐาน',
  'ไว้',
  'ก่อน',
  'ว่า',
  'นายหน้า',
  'ย่อม',
  'ไม่',
  'มีอำนาจ',
  'ที

In [34]:
list_pred_all_wer=[wordcut2(i["generated_text"].strip("assistant\n\n")) for i in pred_list]
list_ans_all_wer=[wordcut2(i) for i in list(dataset["answer"])]

In [35]:
from jiwer import wer,cer # 100%
print("wer: "+str(wer(list_ans_all_wer, list_pred_all_wer)))
print("cer: "+str(cer(list_ans_all_wer, list_pred_all_wer)))

wer: 0.7916802326843351
cer: 0.7322104862833891


In [36]:
from evaluate import load
# Load the ROUGE metric
import evaluate
rouge = evaluate.load('rouge')

In [37]:
results = rouge.compute(predictions=list_pred_all_wer, references=list_ans_all_wer)
print(results)

{'rouge1': 0.5046606645592704, 'rouge2': 0.15032700543425676, 'rougeL': 0.4816149213748217, 'rougeLsum': 0.4846524000828039}


In [38]:
import json

In [39]:
with open("pred-mark4.json","w") as f:
    f.write(json.dumps(pred_list,ensure_ascii=False))